# Зависимости



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install numpy
! pip install pymorphy2[fast]
! pip install nltk


     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 10.7 MB/s 
     |████████████████████████████████| 371 kB 53.3 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=858796 sha256=7e4883e39046e66ee6d012406a23ef6c4a0e994bca0a5890349b00b720dc81a6
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import random
import nltk
import re
import string
import os
import shutil
from nltk.corpus import stopwords
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from typing import List
import nltk
import string
import pymorphy2
import codecs
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='pymorphy2')

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
stopWordsRu = set(stopwords.words("russian")) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
class MaskCreator():

  def __init__(self):
    self.morph = pymorphy2.MorphAnalyzer()
    self.tokenizer = nltk.WordPunctTokenizer()
    self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")
    self.keywords = set()
    for category in ['literature', 'music', 'sport', 'animals']:
      with open("/content/drive/MyDrive/aiijc_sber/true_keywords_nouns_" + category+ ".txt", 'r') as file:
        self.keywords |= set([line.strip() for line in file.readlines()])

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def mask(self, text) -> (str, List[str]):
    masks_dict = []
    tokens = self.tokenizer.tokenize(text.lower())
    tokens_normal = [self.morph.parse(w)[0].normal_form for w in tokens]
    tokens_bigrammed = self.make_bigrams(tokens_normal)
    
    if len(tokens_bigrammed) < len(tokens):
      ind_go = 0
      for i in range(len(tokens_bigrammed)):
        if tokens_normal[ind_go] != tokens_bigrammed[i]:
          tokens = tokens[:ind_go] + [tokens_bigrammed[i]] + tokens[ind_go+2:]
          ind_go += 2
        else:
          ind_go += 1

    for ind, token in enumerate(tokens):
      word = self.morph.parse(token.lower())[0].normal_form
      if word in self.keywords:
        if word not in masks_dict:
          masks_dict.append(word)
        tokens[ind] = 'mask' + str(masks_dict.index(word, 0))
    text = nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(tokens)
    text = text.replace(' .', '.')
    return text, masks_dict

In [22]:
MaskCreator().mask("Глобус - модель земного шара. В морях плавают корабли.")

('глобус - модель земной_шар. в морях плавают mask0.', ['корабль'])